In [23]:
!ls work/out

hard_negs_v3	sneeze_ds_cnn_dynamic.tflite  threshold.txt
norm_stats.npz	sneeze_ds_cnn.keras


In [24]:
!pip -q install librosa soundfile tqdm scikit-learn

## 셀 1. 경로/설정

In [25]:
from pathlib import Path
import random, numpy as np
import tensorflow as tf

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

WORK = Path("/content/work")

ESC50_DIR = WORK / "esc-50"
MS_SNSD_DIR = WORK / "MS-SNSD-NOISE"
SNEEZE_DIR = WORK / "sneeze"
REC_DIR = WORK / "recordings"

# 이미 존재하는 out 폴더를 그대로 사용
OUT_DIR = WORK / "out"
assert OUT_DIR.exists(), f"OUT_DIR not found: {OUT_DIR}"

# out 안의 파일명은 환경마다 다를 수 있으니, 아래 2줄만 out listing 보고 맞춰서 수정
V3_TFLITE = OUT_DIR / "sneeze_ds_cnn_dynamic.tflite"
V3_STATS  = OUT_DIR / "norm_stats.npz"

# v3 keras가 out에 있으면 이어학습 가능
V3_KERAS = OUT_DIR / "sneeze_ds_cnn.keras"

print("OUT_DIR:", OUT_DIR)
print("V3_TFLITE exists:", V3_TFLITE.exists(), V3_TFLITE)
print("V3_STATS exists:", V3_STATS.exists(), V3_STATS)
print("V3_KERAS exists:", V3_KERAS.exists(), V3_KERAS)

# 오디오 규격
SR = 16000
CLIP_SECONDS = 2.0
CLIP_SAMPLES = int(SR * CLIP_SECONDS)

# log-mel 파라미터(지금 실시간 코드와 동일해야 함)
N_MELS = 64
N_FFT = 400
HOP = 160

# 학습 때 쓰던 RMS 정규화 타겟(기존 파이프라인과 같게)
TARGET_RMS = 0.1


OUT_DIR: /content/work/out
V3_TFLITE exists: True /content/work/out/sneeze_ds_cnn_dynamic.tflite
V3_STATS exists: True /content/work/out/norm_stats.npz
V3_KERAS exists: True /content/work/out/sneeze_ds_cnn.keras


## 셀 2. 유틸(로드/2초 고정/특징/정규화)

In [26]:
def rms(x, eps=1e-8):
    x = np.asarray(x, np.float32)
    return float(np.sqrt(np.mean(x*x) + eps))

def normalize_rms(x, target=TARGET_RMS, eps=1e-8):
    x = np.asarray(x, np.float32)
    r = rms(x, eps=eps)
    if r > 1e-6:
        x = x * (target / (r + eps))
    return np.clip(x, -1.0, 1.0).astype(np.float32)

def load_wav_mono_16k(path: Path, sr=SR):
    y, _sr = librosa.load(str(path), sr=sr, mono=True)
    return y.astype(np.float32)

def fix_2s(y, n=CLIP_SAMPLES):
    y = np.asarray(y, np.float32)
    if len(y) >= n:
        return y[:n]
    return np.pad(y, (0, n - len(y))).astype(np.float32)

def logmel(y_16k_2s):
    S = librosa.feature.melspectrogram(
        y=y_16k_2s, sr=SR, n_fft=N_FFT, hop_length=HOP, n_mels=N_MELS, power=2.0
    )
    return np.log(S + 1e-6).T.astype(np.float32)  # (frames, mels)

def clip_from_long(y, start_sec, dur_sec=CLIP_SECONDS, sr=SR):
    s = int(start_sec * sr)
    e = s + int(dur_sec * sr)
    seg = y[s:e]
    return fix_2s(seg)

print("OK")


OK


## 셀 3. v3 norm_stats 로드 + v3 전처리 함수

In [27]:
assert V3_STATS.exists(), f"v3 stats not found: {V3_STATS}"

st = np.load(str(V3_STATS), allow_pickle=True)
mu = st["mu"].astype(np.float32)
sd = st["sd"].astype(np.float32)

# (1,1,mels) -> (mels,)
if mu.ndim == 3 and mu.shape[0] == 1 and mu.shape[1] == 1:
    mu = mu.reshape(-1)
if sd.ndim == 3 and sd.shape[0] == 1 and sd.shape[1] == 1:
    sd = sd.reshape(-1)

# (1,frames,mels) -> (frames,mels)
if mu.ndim == 3 and mu.shape[0] == 1:
    mu = mu[0]
if sd.ndim == 3 and sd.shape[0] == 1:
    sd = sd[0]

def v3_preproc(y16_2s):
    y = fix_2s(y16_2s)
    y = normalize_rms(y)
    f = logmel(y)  # (frames,mels)
    fn = (f - mu) / (sd + 1e-6)
    return fn[None, ..., None].astype(np.float32)  # (1,frames,mels,1)

print("mu shape:", np.shape(mu), "sd shape:", np.shape(sd))


mu shape: (64,) sd shape: (64,)


## 셀 4. v3 TFLite 로더

In [28]:
assert V3_TFLITE.exists(), f"v3 tflite not found: {V3_TFLITE}"

try:
    from tflite_runtime.interpreter import Interpreter as TFLiteInterpreter
except Exception:
    TFLiteInterpreter = tf.lite.Interpreter

class TFLiteModel:
    def __init__(self, path: Path):
        self.interp = TFLiteInterpreter(model_path=str(path))
        self.interp.allocate_tensors()
        self.in_det = self.interp.get_input_details()[0]
        self.out_det = self.interp.get_output_details()[0]

    def predict(self, x):
        self.interp.set_tensor(self.in_det["index"], x)
        self.interp.invoke()
        y = self.interp.get_tensor(self.out_det["index"]).reshape(-1)[0]
        return float(y)

tfl = TFLiteModel(V3_TFLITE)
print("v3 tflite ready")


v3 tflite ready


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


## 셀 5. 하드 네거티브 마이닝

In [29]:
import soundfile as sf
import pandas as pd
from tqdm import tqdm
import librosa

rec_long_files = sorted([p for p in REC_DIR.rglob("*.wav")])
print("recordings wav:", len(rec_long_files))
for p in rec_long_files[:10]:
    print(" -", p)

HARD_DIR = OUT_DIR / "hard_negs_v3"
HARD_DIR.mkdir(parents=True, exist_ok=True)

def mine_hard_negs_from_long(path: Path, topk=60, hop_sec=0.25, min_rms=0.003, min_p=0.50):
    y = load_wav_mono_16k(path)
    if len(y) < CLIP_SAMPLES:
        return []

    hop = int(hop_sec * SR)
    scores = []

    for start in range(0, max(1, len(y) - CLIP_SAMPLES + 1), hop):
        seg = y[start:start+CLIP_SAMPLES]
        seg = fix_2s(seg)
        if rms(seg) < min_rms:
            continue
        x_in = v3_preproc(seg)
        p = tfl.predict(x_in)
        if p < min_p:
            continue
        scores.append((p, start / SR, seg))

    scores.sort(key=lambda x: x[0], reverse=True)

    picked = []
    for p, s, seg in scores:
        if len(picked) >= topk:
            break
        # 2초 간격 이상 떨어진 것만 채택(중복 방지)
        if all(abs(s - s2) >= 2.0 for _, s2, _ in picked):
            picked.append((p, s, seg))
    return picked

hard_meta = []
hard_neg_audio = []

TOPK_PER_FILE = 60
HOP_SEC = 0.25
MIN_RMS = 0.003
MIN_P = 0.50

for p in tqdm(rec_long_files, desc="hard neg mining v3"):
    picked = mine_hard_negs_from_long(p, topk=TOPK_PER_FILE, hop_sec=HOP_SEC, min_rms=MIN_RMS, min_p=MIN_P)
    for score, start_sec, seg in picked:
        out_wav = HARD_DIR / f"hardneg_{p.stem}_{start_sec:.2f}_p{score:.3f}.wav"
        sf.write(str(out_wav), seg, SR)
        hard_neg_audio.append(seg)
        hard_meta.append({
            "source":"hardneg_v3",
            "file": str(p),
            "start": float(start_sec),
            "score": float(score),
            "label": 0,
            "saved": str(out_wav),
            "rms": float(rms(seg)),
        })

print("hard negatives:", len(hard_neg_audio))
pd.DataFrame(hard_meta).sort_values("score", ascending=False).head(10)


recordings wav: 4
 - /content/work/recordings/dish.wav
 - /content/work/recordings/noise1.wav
 - /content/work/recordings/noise2.wav
 - /content/work/recordings/talk.wav


hard neg mining v3: 100%|██████████| 4/4 [01:48<00:00, 27.16s/it]

hard negatives: 211


,source,file,start,score,label,saved,rms
0,hardneg_v3,/content/work/recordings/dish.wav,217.25,0.995522,0,/content/work/out/hard_negs_v3/hardneg_dish_21...,0.063023
60,hardneg_v3,/content/work/recordings/noise1.wav,483.25,0.994170,0,/content/work/out/hard_negs_v3/hardneg_noise1_...,0.072899
61,hardneg_v3,/content/work/recordings/noise1.wav,477.25,0.993641,0,/content/work/out/hard_negs_v3/hardneg_noise1_...,0.083230
62,hardneg_v3,/content/work/recordings/noise1.wav,481.25,0.992340,0,/content/work/out/hard_negs_v3/hardneg_noise1_...,0.074973
63,hardneg_v3,/content/work/recordings/noise1.wav,479.25,0.984472,0,/content/work/out/hard_negs_v3/hardneg_noise1_...,0.090427
64,hardneg_v3,/content/work/recordings/noise1.wav,485.50,0.974088,0,/content/work/out/hard_negs_v3/hardneg_noise1_...,0.057546
65,hardneg_v3,/content/work/recordings/noise1.wav,487.50,0.973766,0,/content/work/out/hard_negs_v3/hardneg_noise1_...,0.075420
1,hardneg_v3,/content/work/recordings/dish.wav,121.75,0.971508,0,/content/work/out/hard_negs_v3/hardneg_dish_12...,0.047617
2,hardneg_v3,/content/work/recordings/dish.wav,41.00,0.971018,0,/content/work/out/hard_negs_v3/hardneg_dish_41...,0.032631
151,hardneg_v3,/content/work/recordings/talk.wav,1504.25,0.970973,0,/content/work/out/hard_negs_v3/hardneg_talk_15...,0.138339


## 셀 6. 기본 학습 데이터 로드(포지티브 + 기존 네거티브) + 하드 네거티브 추가

In [30]:
pos_files = sorted([p for p in SNEEZE_DIR.rglob("*.wav")])
esc_files = sorted([p for p in ESC50_DIR.rglob("*.wav")])
ms_files  = sorted([p for p in MS_SNSD_DIR.rglob("*.wav")])

print("pos:", len(pos_files), "esc:", len(esc_files), "ms:", len(ms_files), "hard:", len(hard_neg_audio))

def sample_neg_from_long_wav(path: Path, n_samples: int, min_rms: float = 0.003):
    y = load_wav_mono_16k(path)
    if len(y) < CLIP_SAMPLES:
        return []
    dur = len(y) / SR
    out = []
    tries = 0
    max_tries = n_samples * 30

    while len(out) < n_samples and tries < max_tries:
        tries += 1
        start = random.uniform(0, max(0.0, dur - CLIP_SECONDS))
        seg = clip_from_long(y, start)
        if rms(seg) < min_rms:
            continue
        out.append((seg, {"source": "recordings_rand", "file": str(path), "start": float(start), "rms": float(rms(seg)), "label": 0}))
    return out

def sample_neg_from_short_wav(path: Path):
    y = load_wav_mono_16k(path)
    seg = fix_2s(y)
    return seg, {"source": "shortwav", "file": str(path), "start": 0.0, "rms": float(rms(seg)), "label": 0}

# 1) 포지티브
pos_audio = []
meta_rows = []
for p in tqdm(pos_files, desc="load pos"):
    y = fix_2s(load_wav_mono_16k(p))
    pos_audio.append(y)
    meta_rows.append({"source":"sneeze", "file": str(p), "start": 0.0, "rms": float(rms(y)), "label": 1})

# 2) 네거티브 기본 풀: esc/ms + recordings 랜덤 + hard
neg_audio = []

# 네거티브 목표: pos*4 정도
POS_N = len(pos_audio)
NEG_TARGET = POS_N * 4

# hard neg는 가능한 많이 넣되, 너무 많으면 pos*2 정도로 제한
HARD_MAX = min(len(hard_neg_audio), POS_N * 2)
hard_sel = hard_neg_audio[:HARD_MAX]
neg_audio.extend(hard_sel)
for i, seg in enumerate(hard_sel):
    meta_rows.append({"source":"hardneg_v3", "file":"(from mining)", "start": 0.0, "rms": float(rms(seg)), "label": 0})

remaining = max(0, NEG_TARGET - len(neg_audio))
print("NEG_TARGET:", NEG_TARGET, "hard_used:", len(hard_sel), "remaining:", remaining)

# 나머지 네거티브는 recordings 랜덤 + esc/ms로 채움
neg_from_rec = int(remaining * 0.50)
neg_from_esc = int(remaining * 0.25)
neg_from_ms  = remaining - neg_from_rec - neg_from_esc

# recordings 랜덤 샘플
if len(rec_long_files) > 0 and neg_from_rec > 0:
    per_file = max(1, neg_from_rec // len(rec_long_files))
    for p in tqdm(rec_long_files, desc="neg rand from recordings"):
        segs = sample_neg_from_long_wav(p, n_samples=per_file, min_rms=0.003)
        for seg, info in segs:
            neg_audio.append(seg)
            meta_rows.append(info)
    # 부족하면 추가
    while len(neg_audio) < len(hard_sel) + neg_from_rec:
        p = random.choice(rec_long_files)
        segs = sample_neg_from_long_wav(p, n_samples=50, min_rms=0.003)
        for seg, info in segs:
            neg_audio.append(seg); meta_rows.append(info)
            if len(neg_audio) >= len(hard_sel) + neg_from_rec:
                break

# esc/ms 샘플
def add_from_pool(pool_files, target_n, source_name):
    if target_n <= 0 or len(pool_files) == 0:
        return
    for p in tqdm(random.sample(pool_files, min(len(pool_files), target_n)), desc=f"neg from {source_name}"):
        seg, info = sample_neg_from_short_wav(p)
        info["source"] = source_name
        neg_audio.append(seg)
        meta_rows.append(info)

add_from_pool(esc_files, neg_from_esc, "esc-50")
add_from_pool(ms_files,  neg_from_ms,  "ms-snsd")

print("pos_audio:", len(pos_audio), "neg_audio:", len(neg_audio))


pos: 968 esc: 2000 ms: 128 hard: 211


load pos: 100%|██████████| 968/968 [00:00<00:00, 1368.58it/s]


NEG_TARGET: 3872 hard_used: 211 remaining: 3661


neg from ms-snsd: 100%|██████████| 128/128 [00:03<00:00, 39.57it/s]

pos_audio: 968 neg_audio: 3084


## 셀 7. 특징 추출(반드시 v3 norm_stats로 정규화)

In [31]:
def feats_list(audio_list):
    feats = []
    for y in tqdm(audio_list, desc="logmel"):
        y = normalize_rms(fix_2s(y))
        f = logmel(y)
        fn = (f - mu) / (sd + 1e-6)
        feats.append(fn.astype(np.float32))
    return feats

pos_feats = feats_list(pos_audio)
neg_feats = feats_list(neg_audio)

X = np.array(pos_feats + neg_feats, dtype=np.float32)  # (N, frames, mels)
y = np.array([1]*len(pos_feats) + [0]*len(neg_feats), dtype=np.int64)

X = X[..., None]  # (N, frames, mels, 1)
print("X:", X.shape, "y:", y.shape, "pos:", int(y.sum()), "neg:", int((y==0).sum()))


logmel: 100%|██████████| 3084/3084 [00:11<00:00, 264.45it/s]


X: (4052, 201, 64, 1) y: (4052,) pos: 968 neg: 3084


## 셀 8. train/val/test 분할

In [35]:
from sklearn.model_selection import train_test_split

idx = np.arange(len(y))
idx_train, idx_tmp, y_train, y_tmp = train_test_split(idx, y, test_size=0.30, random_state=SEED, stratify=y)
idx_val, idx_test, y_val, y_test = train_test_split(idx_tmp, y_tmp, test_size=0.50, random_state=SEED, stratify=y_tmp)

X_train, X_val, X_test = X[idx_train], X[idx_val], X[idx_test]
y_train, y_val, y_test = y[idx_train], y[idx_val], y[idx_test]

print("train:", X_train.shape, "pos_rate:", float(y_train.mean()))
print("val:", X_val.shape, "pos_rate:", float(y_val.mean()))
print("test:", X_test.shape, "pos_rate:", float(y_test.mean()))


train: (2836, 201, 64, 1) pos_rate: 0.2390691114245416
val: (608, 201, 64, 1) pos_rate: 0.23848684210526316
test: (608, 201, 64, 1) pos_rate: 0.23848684210526316


## 셀 9. 모델 정의(DS-CNN) + v3에서 이어학습

In [36]:
def ds_conv_block(x, filters, k=3, s=1, dropout=0.0):
    x = tf.keras.layers.DepthwiseConv2D((k,k), strides=(s,s), padding="same", use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(filters, (1,1), padding="same", use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    if dropout > 0:
        x = tf.keras.layers.Dropout(dropout)(x)
    return x

def build_model(input_shape):
    inp = tf.keras.Input(shape=input_shape)

    x = tf.keras.layers.Conv2D(32, (3,3), strides=(2,2), padding="same", use_bias=False)(inp)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = ds_conv_block(x, 64,  k=3, s=1, dropout=0.05)
    x = ds_conv_block(x, 96,  k=3, s=2, dropout=0.05)
    x = ds_conv_block(x, 128, k=3, s=1, dropout=0.05)
    x = ds_conv_block(x, 160, k=3, s=2, dropout=0.05)
    x = ds_conv_block(x, 192, k=3, s=1, dropout=0.05)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    return tf.keras.Model(inp, out)

if V3_KERAS.exists():
    model = tf.keras.models.load_model(str(V3_KERAS))
    print("Loaded v3 keras:", V3_KERAS)
else:
    model = build_model(X_train.shape[1:])
    print("Built new model (v3-like)")

model.summary()


Loaded v3 keras: /content/work/out/sneeze_ds_cnn.keras


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 201, 64, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d                │ (None, 201, 64, 16)    │            25 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 201, 64, 16)    │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 201, 64, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 100, 32, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_1              │ (None, 100, 32, 24)    │           528 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100, 32, 24)    │            96 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 100, 32, 24)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 50, 16, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_2              │ (None, 50, 16, 32)     │           984 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 50, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 50, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 25, 8, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_3              │ (None, 25, 8, 48)      │         1,824 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 25, 8, 48)      │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 25, 8, 48)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 4, 48)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 48)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            49 │
└─────────────────────────────────┴────────────────────────┴─────────────

 Total params: 11,192 (43.72 KB)

 Trainable params: 3,650 (14.26 KB)

 Non-trainable params: 240 (960.00 B)

 Optimizer params: 7,302 (28.53 KB)

## 셀 10. 학습(v3.1)

In [37]:
pos = int((y_train==1).sum())
neg = int((y_train==0).sum())
class_weight = {0: 1.0, 1: neg / (pos + 1e-6)}
print("class_weight:", class_weight)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
    ],
)

cb = [
    tf.keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.5, patience=2, min_lr=1e-5),
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    class_weight=class_weight,
    callbacks=cb,
    verbose=1
)


class_weight: {0: 1.0, 1: 3.182890850762698}
Epoch 1/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 29s 369ms/step - auc: 0.9360 - loss: 0.5120 - precision: 0.6888 - recall: 0.8781 - val_auc: 0.9567 - val_loss: 0.1985 - val_precision: 0.8690 - val_recall: 0.8690 - learning_rate: 0.0010
Epoch 2/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.9771 - loss: 0.2980 - precision: 0.8173 - recall: 0.9437 - val_auc: 0.9632 - val_loss: 0.1872 - val_precision: 0.9058 - val_recall: 0.8621 - learning_rate: 0.0010
Epoch 3/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.9797 - loss: 0.2754 - precision: 0.8191 - recall: 0.9464 - val_auc: 0.9671 - val_loss: 0.1860 - val_precision: 0.8658 - val_recall: 0.8897 - learning_rate: 0.0010
Epoch 4/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.9825 - loss: 0.2543 - precision: 0.8402 - recall: 0.9627 - val_auc: 0.9735 - val_loss: 0.1672 - val_precision: 0.8828 - val_recall: 0.8828 - learning_rate: 0.0010
Epoch 5/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.9

## 셀 11. 평가 + threshold 선택(오탐 최소 기준)

In [38]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve


val_prob = model.predict(X_val, verbose=0).reshape(-1)
test_prob = model.predict(X_test, verbose=0).reshape(-1)

print("val auc:", roc_auc_score(y_val, val_prob))
print("test auc:", roc_auc_score(y_test, test_prob))

fpr, tpr, thr = roc_curve(y_val, val_prob)

target_fpr = 0.01  # 1% 오탐 목표. 필요하면 0.005로 더 빡세게.
candidates = [(t, tp, fp) for t, tp, fp in zip(thr, tpr, fpr) if fp <= target_fpr]
if len(candidates) == 0:
    best_thr = 0.5
    best_tp = float(tpr[np.argmin(np.abs(thr-0.5))])
    best_fp = float(fpr[np.argmin(np.abs(thr-0.5))])
else:
    best_thr, best_tp, best_fp = sorted(candidates, key=lambda x: x[1], reverse=True)[0]

print("chosen threshold:", float(best_thr), "val_tpr:", float(best_tp), "val_fpr:", float(best_fp))

test_pred = (test_prob >= best_thr).astype(int)
cm = confusion_matrix(y_test, test_pred)
print("confusion_matrix:\n", cm)
print(classification_report(y_test, test_pred, digits=4))


val auc: 0.9817978699635063
test auc: 0.9919565055485215
chosen threshold: 0.9741973876953125 val_tpr: 0.6068965517241379 val_fpr: 0.008639308855291577
confusion_matrix:
 [[461   2]
 [ 63  82]]
              precision    recall  f1-score   support

           0     0.8798    0.9957    0.9341       463
           1     0.9762    0.5655    0.7162       145

    accuracy                         0.8931       608
   macro avg     0.9280    0.7806    0.8252       608
weighted avg     0.9028    0.8931    0.8822       608



## 셀 12. v3.1 저장(Keras + TFLite)

In [39]:
SAVE_KERAS = OUT_DIR / "v3_1_model.keras"
model.save(str(SAVE_KERAS))
print("saved:", SAVE_KERAS)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

SAVE_TFLITE = OUT_DIR / "v3_1_model.tflite"
SAVE_TFLITE.write_bytes(tflite_model)
print("saved:", SAVE_TFLITE)

# threshold도 같이 기록해두면 실전에서 편합니다.
thr_path = OUT_DIR / "v3_1_threshold.txt"
thr_path.write_text(str(float(best_thr)))
print("saved:", thr_path)

print("OUT_DIR:", OUT_DIR)
for p in sorted(OUT_DIR.rglob("*")):
    if p.is_file():
        print(" -", p.relative_to(OUT_DIR))
print("hard neg dir:", HARD_DIR)
print("hard neg files:", len(list(HARD_DIR.rglob("*.wav"))))


saved: /content/work/out/v3_1_model.keras
Saved artifact at '/tmp/tmpjlwz0der'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 201, 64, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  137462711777808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137462711778000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137462711779152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137462711778192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137462711777040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137462711778768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137462712451920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137462712452304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137462712453264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137462712453456: TensorSpec(shape=(), dtype=

In [43]:
!ls work/out

hard_negs_v3		      sneeze_ds_cnn.keras  v3_1_model.tflite
norm_stats.npz		      threshold.txt	   v3_1_threshold.txt
sneeze_ds_cnn_dynamic.tflite  v3_1_model.keras


In [44]:
!cp -r /content/work/out /content/drive/MyDrive/sneeze_models/